In [1]:
# Step 0: Install necessary libraries (if not already installed)
!pip install --upgrade gspread gspread_dataframe google-auth

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gspread
from gspread_dataframe import get_as_dataframe
import datetime
import re

# Authenticate your Google account (Colab will prompt you)
from google.colab import auth
auth.authenticate_user()

# Get the default credentials and authorize gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# --- User Configuration ---
# Replace with your actual Google Sheet ID (the long string in your sheet URL)
sheet_id = "1VqnpplRwT7m-f0DV5LW62DWh-bDEGQe2LDJpdQkbzKc"

# Open the spreadsheet by its key (ID)
spreadsheet = gc.open_by_key(sheet_id)

# Open a specific worksheet (for example, "MyEBirdData")
worksheet = spreadsheet.worksheet("MyEBirdData")

# Read the worksheet into a DataFrame (optional)
df_raw = get_as_dataframe(worksheet, evaluate_formulas=True, header=0)
print("Data from Google Sheet:")
print(df_raw.head())


Data from Google Sheet:
  Submission ID               Common Name       Scientific Name  \
0    S212160259    Spotted Whistling-Duck   Dendrocygna guttata   
1    S212160267    Fulvous Whistling-Duck   Dendrocygna bicolor   
2    S212160278  Wandering Whistling-Duck   Dendrocygna arcuata   
3    S158487375  Wandering Whistling-Duck   Dendrocygna arcuata   
4    S156606957     Lesser Whistling-Duck  Dendrocygna javanica   

   Taxonomic Order Count State/Province        County Location ID  \
0            229.0    19          ID-MA  Maluku Utara   L38768005   
1            239.0     8          MM-04           NaN   L39743759   
2            242.0    16         PH-ILI           NaN    L5530663   
3            242.0    40          MY-12           NaN    L3081815   
4            248.0   122          KH-21           NaN   L28280274   

                                            Location   Latitude  ...  \
0                                       Pasar Labuha  -0.635288  ...   
1  Paleik lake

1. Reading the Sheets into DataFrames


In [ ]:
# Open the sheet by ID
spreadsheet = gc.open_by_key(sheet_id)

# Read each worksheet into a DataFrame
dfs = {}
for key, ws_name in worksheet_names.items():
    worksheet = spreadsheet.worksheet(ws_name)
    # Use get_as_dataframe; set header=0 (first row) and skip rows if necessary.
    df = get_as_dataframe(worksheet, evaluate_formulas=True, header=0)
    # Drop rows that are completely empty (if any)
    df.dropna(how='all', inplace=True)
    dfs[key] = df

# For convenience assign each DataFrame to a variable
df_raw = dfs["raw_data"]
df_dec = dfs["dec_data"]
df_phil = dfs["phil_raw"]
df_pop = dfs["populations"]

# Display basic info
print("Raw data shape:", df_raw.shape)
print("Dec data shape:", df_dec.shape)
print("Philippines raw data shape:", df_phil.shape)
print("Populations data shape:", df_pop.shape)


2. Data Cleaning & Converting Types


In [ ]:
# Let’s assume your column for Date is labeled "Date" (case sensitive)
for df in [df_raw, df_dec, df_phil]:
    if "Date" in df.columns:
        # Convert Date column to datetime. You may need to adjust the format.
        df["Date"] = pd.to_datetime(df["Date"], errors='coerce')

# For the raw Philippines data, also ensure Count is numeric.
if "Count" in df_phil.columns:
    df_phil["Count"] = pd.to_numeric(df_phil["Count"], errors='coerce')


3. Build a Population Lookup Data Structure


In [ ]:
# We assume df_pop has columns such as:
# "Scientific Name", "Common Name", "Red List", "Population Name", "1% threshold", etc.
# (If "1% threshold" is not the actual header, adjust accordingly.)
pop_lookup = {}

# Clean column names (strip spaces)
df_pop.columns = [col.strip() for col in df_pop.columns]

for idx, row in df_pop.iterrows():
    sci_name = row["Scientific Name"].strip() if pd.notnull(row["Scientific Name"]) else None
    if sci_name is None:
        continue
    # Create a dictionary with the fields we want; add more fields as needed.
    pop_data = {
        "common_name": row.get("Common Name", None),
        "red_list": row.get("Red List", None),
        "population_name": row.get("Population Name", None),
        "iucn": row.get("Red List", None),  # assuming IUCN status is in the Red List column
        "one_percent_threshold": row.get("1% threshold", np.nan)  # adjust column name if needed
        # you can add additional fields such as breeding/nonbreeding ranges here
    }
    pop_lookup.setdefault(sci_name, []).append(pop_data)

# For debug: print one example
example_key = list(pop_lookup.keys())[0]
print("Example populations for", example_key, ":", pop_lookup[example_key])


4. Define a Waterbird List


In [ ]:
# For example, let’s assume you have a list of waterbird common names:
waterbird_common_names = {
    "Spotted Whistling-Duck", "Bar-tailed Godwit", "Asian Emerald Dove", "Barn Swallow",
    # ... add additional species names here
}

# Alternatively, if you prefer to match by scientific name, create that set:
waterbird_scientific_names = {
    "Dendrocygna guttata", "Limosa lapponica", # etc.
}

def is_waterbird(row):
    # Decide whether to use common name or scientific name
    common = row.get("Common Name", "")
    sci = row.get("Scientific Name", "")
    return (common in waterbird_common_names) or (sci in waterbird_scientific_names)


5. Create a Summary Table by Site


In [ ]:
def summarize_site(df, pop_lookup):
    # We assume that the site name is in the "Location" column.
    summary_rows = []
    # Get unique site names
    sites = df["Location"].unique()

    for site in sites:
        # Filter the data for the current site
        df_site = df[df["Location"] == site]

        # Get unique species present in this site (we use both Common and Scientific Names)
        # Here we assume that each record has a "Common Name" and "Scientific Name"
        species = df_site[["Common Name", "Scientific Name"]].drop_duplicates()

        for _, sp_row in species.iterrows():
            common = sp_row["Common Name"]
            sci = sp_row["Scientific Name"]
            # Filter rows corresponding to this species at the current site
            df_species = df_site[(df_site["Common Name"] == common) & (df_site["Scientific Name"] == sci)]

            total_count = df_species["Count"].sum()
            num_trips = df_species["Date"].nunique()
            avg_count = total_count / num_trips if num_trips > 0 else np.nan
            max_count = df_species["Count"].max()
            waterbird_flag = is_waterbird(sp_row)

            # Look up population data based on scientific name.
            pop_info = pop_lookup.get(sci, None)
            if pop_info:
                # If there are multiple populations, we list them all (or you could choose one)
                pop_names = "; ".join([str(pi.get("population_name", "")) for pi in pop_info])
                iucn = pop_info[0].get("iucn", None)  # for example, take the first pop's IUCN
                one_percent = pop_info[0].get("one_percent_threshold", np.nan)
            else:
                pop_names = ""
                iucn = ""
                one_percent = np.nan

            # Check if total count exceeds the 1% threshold (if threshold is available)
            exceeds_threshold = (total_count >= one_percent) if not np.isnan(one_percent) else False

            summary_rows.append({
                "Site": site,
                "Common Name": common,
                "Scientific Name": sci,
                "isWaterbird": waterbird_flag,
                "Population(s)": pop_names,
                "IUCN": iucn,
                "Count (this month)": total_count,
                "No. of trips": num_trips,
                "Average count": round(avg_count, 2) if not np.isnan(avg_count) else np.nan,
                "Max count": max_count,
                "1% threshold": one_percent,
                "Equal or exceed 1%": exceeds_threshold
            })

    summary_df = pd.DataFrame(summary_rows)
    return summary_df

# Generate the summary table for Philippines sites:
site_summary = summarize_site(df_phil, pop_lookup)
print(site_summary.head())


# Group by site to compute totals:
site_totals = site_summary.groupby("Site").agg({
    "Count (this month)": "sum",
    "isWaterbird": lambda x: np.sum(site_summary.loc[x.index, "Count (this month)"])  # sum count for waterbirds
}).rename(columns={"Count (this month)":"Total Count", "isWaterbird": "Total Waterbird Count"})

print(site_totals)


6. Generating Pie Charts


In [ ]:
# Define colours for IUCN categories:
iucn_colors = {
    "LC": "#60C659",
    "NT": "#60C659",  # adjust if needed
    "VU": "#60C659",
    "EN": "#FC7F3F",
    "CR": "#D81E05"
}

def plot_site_pie_chart(df, site, iucn_colors):
    # Filter summary rows for the site and only waterbirds
    df_site = df[(df["Site"] == site) & (df["isWaterbird"] == True)]
    if df_site.empty:
        print(f"No waterbird data for site {site}")
        return

    # We will sum the counts per IUCN category.
    # Some rows might have blank IUCN; fill them as "Unknown"
    df_site["IUCN_clean"] = df_site["IUCN"].fillna("Unknown")
    counts = df_site.groupby("IUCN_clean")["Count (this month)"].sum()

    # Use colours from our dictionary (or default if not found)
    colors = [iucn_colors.get(cat, "#cccccc") for cat in counts.index]

    plt.figure(figsize=(6,6))
    plt.pie(counts, labels=counts.index, autopct="%1.1f%%", colors=colors, startangle=90)
    plt.title(f"IUCN Category Breakdown for {site}\n(Total waterbird count = {df_site['Count (this month)'].sum()})")
    plt.show()

# Example: plot for one site
for site in site_summary["Site"].unique():
    plot_site_pie_chart(site_summary, site, iucn_colors)
